<a href="https://colab.research.google.com/github/jgamel/learn_n_dev/blob/PySpark/Linear_Regression_Pyspark_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark Running a Regression Example

### Setting Up Pyspark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
!wget -q http://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz 
!tar xf spark-3.2.1-bin-hadoop2.7.tgz


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop2.7/"

In [3]:
!pip install findspark

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
spark

# Linear Regression Model


Linear Regression model is one the oldest and widely used machine learning approach which assumes a relationship between dependent and independent variables. For example, a modeler might want to predict the forecast of the rain based on the humidity ratio. Linear Regression consists of the best fitting line through the scattered points on the graph and the best fitting line is known as the regression line.

The goal of this exercise to predict the housing prices by the given features. Let's predict the prices of the Boston Housing dataset by considering MEDV as the output variable and all the other variables as input.

Download the dataset from [here](https://github.com/asifahmed90/pyspark-ML-in-Colab/blob/master/BostonHousing.csv) and keep it somewhere on your computer. Load the dataset into your Colab directory from your local system:

Upload your file from your desktop if you want to:

In [10]:
from google.colab import files
files.upload()

{}

Check if your file or directory is listed

In [11]:
!ls

sample_data  spark-3.2.1-bin-hadoop2.7	spark-3.2.1-bin-hadoop2.7.tgz


For our linear regression model we need to import two modules from Pyspark i.e. Vector Assembler and Linear Regression. Vector Assembler is a transformer that assembles all the features into one vector from multiple columns that contain type double. We could have used StringIndexer if any of our columns contains string values to convert it into numeric values. Luckily, the BostonHousing dataset only contains double values, so we don't need to worry about StringIndexer for now.

In [12]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

Load Data

In [13]:
!wget --continue https://raw.githubusercontent.com/jgamel/learn_n_dev/input_data/BostonHousing.csv -O /tmp/BostonHousing.csv

--2022-05-03 22:04:02--  https://raw.githubusercontent.com/jgamel/learn_n_dev/input_data/BostonHousing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35735 (35K) [text/plain]
Saving to: ‘/tmp/BostonHousing.csv’

/tmp/BostonHousing. 100%[===================>]  34.90K  --.-KB/s    in 0.004s  

2022-05-03 22:04:02 (9.00 MB/s) - ‘/tmp/BostonHousing.csv’ saved [35735/35735]



In [14]:
dataset = spark.read.csv('/tmp/BostonHousing.csv',inferSchema=True, header =True)

Notice that we used InferSchema inside read.csv mofule. InferSchema enables us to infer automatically different data types for each column.

Let us print look into the dataset to see the data types of each column:

In [15]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



Next step is to convert all the features from different columns into a single column and let's call this new vector column as 'Attributes' in the outputCol.

In [16]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","medv")

finalized_data.show(5)

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
+--------------------+----+
only showing top 5 rows



In [17]:
finalized_data.count()

506

Here, 'Attributes' are in the input features from all the columns and 'medv' is the target column. Next, we should split the training and testing data according to our dataset (0.8 and 0.2 in this case).

In [18]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.00906,90.0,2.9...|32.2| 31.47719506231376|
|[0.01096,55.0,2.2...|22.0|27.154166304525106|
|[0.01301,35.0,1.5...|32.7|30.071393866249323|
|[0.01311,90.0,1.2...|35.4|31.257349949581965|
|[0.0136,75.0,4.0,...|18.9|15.299139757616295|
|[0.01501,80.0,2.0...|24.5| 27.51121233301368|
|[0.01501,90.0,1.2...|50.0| 44.59871745040547|
|[0.01538,90.0,3.7...|44.0| 37.40640768721187|
|[0.02055,85.0,0.7...|24.7|24.675918403713275|
|[0.02763,75.0,2.9...|30.8|31.520976246135767|
|[0.03041,0.0,5.19...|18.5|19.512958113602775|
|[0.03502,80.0,4.9...|28.5|33.603903012015756|
|[0.03615,80.0,4.9...|27.9| 32.04676838583227|
|[0.03738,0.0,5.19...|20.7|21.811330209961895|
|[0.04527,0.0,11.9...|20.6|22.454482120098845|
|[0.0536,21.0,5.64...|25.0|27.699246556062988|
|[0.05644,40.0,6.4...|32.4| 36.23187724288983|
|[0.06047,0.0,2.46...|29.6|24.955067905807844|
|[0.06617,0.0

We can also print the coefficient and intercept of the regression model by using the following command:

In [19]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.11, 0.0467, 0.0076, 2.7191, -16.9979, 3.8065, -0.0049, -1.5168, 0.3328, -0.0135, -0.9036, 0.0093, -0.5108])
The Intercept of the model is : 35.925539


### Basic Statistical Analysis

Once we are done with the basic linear regression operation, we can go a bit further and analyze our model statistically by importing RegressionEvaluator module from Pyspark.

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)



spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 4.387
MSE: 19.242
MAE: 3.165
r2: 0.778
